<a href="https://colab.research.google.com/github/kaniska/covid-19-hackathon/blob/master/social-network-mining/BigDataTweetAnalysisApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

https://github.com/JohnSnowLabs/spark-nlp-workshop/tree/master/jupyter

Colab Notebook
https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/quick_start_google_colab.ipynb

Jupyter Notebook using pandas pickle
https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/clinical_text_classification/2.Adverse_Effect_Classification.ipynb

https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/Spark%20NLP%20Healthcare%20Training%20-%20April%202020.pdf

https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/clinical_text_classification/2.Adverse_Effect_Classification.ipynb

Spark-NLP Databrics
https://johnsnowlabs.github.io/spark-nlp-workshop/databricks/index.html#Getting%20Started.html
https://github.com/JohnSnowLabs/spark-nlp-workshop/tree/master/databricks

https://nlp.johnsnowlabs.com/docs/en/install#python

#Natural Language Processing with PySpark and Spark-NLP
https://towardsdatascience.com/natural-language-processing-with-pyspark-and-spark-nlp-b5b29f8faba

#More code to run spark-nlp in clouddataproc
https://github.com/GoogleCloudDataproc/cloud-dataproc/blob/master/codelabs/spark-nlp/topic_model.py



In [0]:
import os
# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! update-alternatives --config java
! java -version

# Install pyspark
! pip install --ignore-installed pyspark==2.4.4

In [0]:
#/usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
! pip install --ignore-installed spark-nlp==2.4.5

In [0]:
import sparknlp
spark = sparknlp.start()

print("Spark NLP version")
sparknlp.version()
print("Apache Spark version")
spark.version

Alternate way of initializing spark with sparknlp

spark = SparkSession.builder \
    .appName("ner")\
    .master("local[4]")\
    .config("spark.driver.memory","8G")\
    .config("spark.driver.maxResultSize", "2G") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:2.4.4")\
    .config("spark.kryoserializer.buffer.max", "500m")\
    .getOrCreate()

In [0]:
from sparknlp.pretrained import PretrainedPipeline

In [0]:
pipeline1 = PretrainedPipeline('recognize_entities_dl', 'en')

In [0]:
annotation = pipeline1.annotate("covid19 PPE available Kaiser Permamnente Medical Fremont")
print(annotation['ner'])

Tutorial - https://nlp.johnsnowlabs.com/docs/en/pipelines#recognize_entities_dl

In [0]:
from pyspark.sql import *

SampleData = Row("id", "text")
sample1 = SampleData(1, "Google has announced the release of a beta version of the popular TensorFlow machine learning library")
sample2 = SampleData(2, "Donald John Trump (born June 14, 1946) is the 45th and current president of the United States")
sample3 = SampleData(3,"covid19 PPE available Kaiser Permamnente Medical Fremont")
samplesSeq = [sample1, sample2, sample3]
sampleDF = spark.createDataFrame(samplesSeq)

#SparkSession.createDataFrame(rows)
#pipeline1.annotate(...)
annotation = pipeline1.transform(sampleDF)
#print(annotation['ner'])
annotation.select("entities.result").show(100)

In [0]:
pipeline2 = PretrainedPipeline('analyze_sentiment', 'en')

In [0]:
result = pipeline2.annotate('I am hopeful that we shall win over covid19 ')
print(result['sentiment'])

In [0]:
# spark-nlp components. Each one is incorporated into our pipeline.
from sparknlp.annotator import Lemmatizer, Stemmer, Tokenizer, Normalizer
from sparknlp.base import DocumentAssembler, Finisher

# A Spark Session is how we interact with Spark SQL to create Dataframes
from pyspark.sql import SparkSession

# These allow us to create a schema for our data
from pyspark.sql.types import StructField, StructType, StringType, LongType

# Spark Pipelines allow us to sequentially add components such as transformers 
from pyspark.ml import Pipeline

# These are components we will incorporate into our pipeline.
from pyspark.ml.feature import StopWordsRemover, CountVectorizer, IDF

# LDA is our model of choice for topic modeling
from pyspark.ml.clustering import LDA

# Some transformers require the usage of other Spark ML functions. We import them here
from pyspark.sql.functions import col, lit, concat

# This will help catch some PySpark errors
from pyspark.sql.utils import AnalysisException

In [0]:
# Now, we begin assembling our pipeline. Each component here is used to some transformation to the data.
# The Document Assembler takes the raw text data and convert it into a format that can
# be tokenized. It becomes one of spark-nlp native object types, the "Document".
document_assembler = DocumentAssembler().setInputCol("text").setOutputCol("document")

# The Tokenizer takes data that is of the "Document" type and tokenizes it. 
# While slightly more involved than this, this is effectively taking a string and splitting
# it along ths spaces, so each word is its own string. The data then becomes the 
# spark-nlp native type "Token".
tokenizer = Tokenizer().setInputCols(["document"]).setOutputCol("token")

# The Normalizer will group words together based on similar semantic meaning. 
normalizer = Normalizer().setInputCols(["token"]).setOutputCol("normalizer")

# The Stemmer takes objects of class "Token" and converts the words into their
# root meaning. For instance, the words "cars", "cars'" and "car's" would all be replaced
# with the word "car".
stemmer = Stemmer().setInputCols(["normalizer"]).setOutputCol("stem")

# The Finisher signals to spark-nlp allows us to access the data outside of spark-nlp
# components. For instance, we can now feed the data into components from Spark MLlib. 
finisher = Finisher().setInputCols(["stem"]).setOutputCols(["to_spark"]).setValueSplitSymbol(" ")

# Stopwords are common words that generally don't add much detail to the meaning
# of a body of text. In English, these are mostly "articles" such as the words "the"
# and "of".
stopword_remover = StopWordsRemover(inputCol="to_spark", outputCol="filtered")

# Here we implement TF-IDF as an input to our LDA model. CountVectorizer (TF) keeps track
# of the vocabulary that's being created so we can map our topics back to their
# corresponding words.
# TF (term frequency) creates a matrix that counts how many times each word in the
# vocabulary appears in each body of text. This then gives each word a weight based
# on its frequency.
tf = CountVectorizer(inputCol="filtered", outputCol="raw_features")

# Here we implement the IDF portion. IDF (Inverse document frequency) reduces 
# the weights of commonly-appearing words. 
idf = IDF(inputCol="raw_features", outputCol="features")

# LDA creates a statistical representation of how frequently words appear 
# together in order to create "topics" or groups of commonly appearing words.
lda = LDA(k=10, maxIter=10)

# We add all of the transformers into a Pipeline object. Each transformer
# will execute in the ordered provided to the "stages" parameter
pipeline = Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        normalizer,        
        stemmer,
        finisher,
        stopword_remover,
        tf,
        idf,
        lda
    ]
)